In [1]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from io import BytesIO
import xmltodict
import requests
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

from astropy.table import Table
from astropy.table import QTable
import astropy_healpix as ah
import astropy.units as u
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess.cosmology import dVC_dVL_for_DL
from astropy.cosmology import Planck15 as cosmo, z_at_value

from ligo.gracedb.rest import GraceDb
g = GraceDb()

# O4a events

options to get all significan O4a mergers, or the 5 mergers considered in O4a paper

In [4]:
# use gracedb api call to get O4a significant superevents, but some of these have been retracted.

event_iterator = g.superevents('runid: O4a SIGNIF_LOCKED')
graceids = [superevent['superevent_id'] for superevent in event_iterator]

print (len(graceids), 'significant superevents in O4a')

89 significant superevents in O4a


In [5]:
responses = [g.superevent(id) for id in graceids]
data = [r.json() for r in responses]

In [4]:
def get_gcn_urls (ids, files):
    """
    remove retracted events
    get the most recent file for each superevent
    """ 

    superevent_files = [i['links']['files'] for i in files]

    event_files = [g.files(graceid).json() for graceid in ids]

    file = ['none' if any('etraction' in s for s in list(files))
            else id+'-5-Update.xml,0' if id+'-5-Update.xml,0' in list(files)
            else id+'-5-Update.xml' if id+'-5-Update.xml' in list(files)
            else id+'-4-Update.xml,0' if id+'-4-Update.xml,0' in list(files)
            else id+'-4-Update.xml' if id+'-4-Update.xml' in list(files)
            else id+'-3-Update.xml,0' if id+'-3-Update.xml,0' in list(files)
            else id+'-2-Update.xml,0' if id+'-2-Update.xml,0' in list(files)
            else id+'-4-Initial.xml,0' if id+'-4-Initial.xml,0' in list(files) 
            else id+'-3-Initial.xml,0' if id+'-3-Initial.xml,0' in list(files)
            else id+'-2-Initial.xml,0' if id+'-2-Initial.xml,0' in list(files)
            else id+'-2-Preliminary.xml,0' if id+'-2-Preliminary.xml,0' in list(files)
            else 'none' for files, id in zip(event_files, graceids)]

    urls = [i+j for i,j in zip(superevent_files, file)]

    [print(x) for x in urls if "none" in x]
    urls_save = [x for x in urls if "none" not in x]
    
    return(urls_save)

def get_params(xml_urls):
    """
    get superevent_id and skymap from event files
    """

    try:
        response = requests.get(xml_urls)
        dict=xmltodict.parse(response.text)
        superevent_id  = [item['@value'] for item in dict['voe:VOEvent']['What']['Param'] if item.get('@name') == 'GraceID'][0]
        skymap_url = [item['Param']['@value'] for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'GW_SKYMAP'][0]
        classification = [item for item in dict['voe:VOEvent']['What']['Group'] if item.get('@name') == 'Classification']
        prob_bns = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'BNS'][0])  
        prob_nsbh = float([item['@value'] for item in classification[0]['Param'] if item.get('@name') == 'NSBH'][0])  
        skymap_response = requests.get(skymap_url)
        skymap_bytes = skymap_response.content
        skymap = Table.read(BytesIO(skymap_bytes))
        return superevent_id, skymap_bytes, skymap, prob_bns, prob_nsbh
    
    except:
        print (xml_urls)

In [7]:
#the printed urls are retracted events or my function failed to get the correct file

gcn_urls = get_gcn_urls (graceids, data)

https://gracedb.ligo.org/api/superevents/S231112ag/files/none
https://gracedb.ligo.org/api/superevents/S230830b/files/none
https://gracedb.ligo.org/api/superevents/S230808i/files/none
https://gracedb.ligo.org/api/superevents/S230715bw/files/none
https://gracedb.ligo.org/api/superevents/S230712a/files/none
https://gracedb.ligo.org/api/superevents/S230708bi/files/none
https://gracedb.ligo.org/api/superevents/S230622ba/files/none


In [ ]:
#returns event id, skymap bytes, skymap table, prob bns, prob nsbh    

params = [get_params(url) for url in gcn_urls]

In [9]:
# get subset of mergers that are potential BNS or NSBH

ns_events = [i for i in params if i[3] + i[4] > 0.1]
ids = [i[0] for i in ns_events] 
print(f'{len(ns_events)} events with prob > 0.1 for NSBH or BNS: {ids}') 

3 events with prob > 0.1 for NSBH or BNS: ['S230731an', 'S230627c', 'S230529ay']


In [5]:
# mergers triggered on in the O4a paper (https://arxiv.org/abs/2405.12403)

O4a_urls = ['https://gracedb.ligo.org/api/superevents/S230518h/files/S230518h-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230529ay/files/S230529ay-5-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230627c/files/S230627c-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S230731an/files/S230731an-4-Update.xml,0',
            'https://gracedb.ligo.org/api/superevents/S231113bw/files/S231113bw-4-Update.xml,0',
            ]

In [6]:
O4a_params = [get_params(url) for url in O4a_urls]

# natalya's rates calculation

In [40]:
def calculate_sky_area_90_percent(skymap, skymap_bytes, target_probability=0.1):
    m, meta = read_sky_map(skymap_bytes)
    #skymap = Table.read(skymap_file)
    
    nside = meta.get('nside', hp.get_nside(m))
    
    sorted_indices = np.argsort(m)[::-1]
    sorted_prob_density = m[sorted_indices]
    cumulative_prob = np.cumsum(sorted_prob_density)
    
    index_target_percent = np.where(cumulative_prob >= target_probability)[0][0]
    pixels_in_target_percent = sorted_indices[:index_target_percent+1]
    
    theta, phi = hp.pix2ang(nside, pixels_in_target_percent)
    ra_deg = np.degrees(phi)
    dec_deg = np.degrees(0.5 * np.pi - theta)
    
    uniq_values = skymap['UNIQ']
    
    level, ipix = ah.uniq_to_level_ipix(uniq_values)
    nside_sky = ah.level_to_nside(level)
    ra_sky, dec_sky = ah.healpix_to_lonlat(ipix, nside_sky, order='nested')
    
    matched_indices = []
    for ra, dec in zip(ra_deg, dec_deg):
        matched = np.where((np.abs(ra_sky.deg - ra) < 1e-6) & (np.abs(dec_sky.deg - dec) < 1e-6))[0]
        if len(matched) > 0:
            matched_indices.extend(matched)
    
    skymap_pixels_in_target_percent = skymap[matched_indices]
    
    pixel_area = hp.nside2pixarea(nside)
    total_area_target_percent = len(pixels_in_target_percent) * pixel_area
    total_area_target_percent_deg2 = total_area_target_percent * (180 / np.pi)**2
    
    plot_skymap_with_90_percent_region(m, nside, pixels_in_target_percent)
    
    return total_area_target_percent_deg2, skymap_pixels_in_target_percent, nside

def calculate_pixel_volumes(skymap, skymap_bytes, target_probability=0.1):
    m, meta = read_sky_map(skymap_bytes)
    #skymap = Table.read(skymap_file)
    
    nside = meta.get('nside', hp.get_nside(m))
    
    sorted_indices = np.argsort(m)[::-1]
    sorted_prob_density = m[sorted_indices]
    cumulative_prob = np.cumsum(sorted_prob_density)
    
    index_target_percent = np.where(cumulative_prob >= target_probability)[0][0]
    pixels_in_target_percent = sorted_indices[:index_target_percent+1]
    
    theta, phi = hp.pix2ang(nside, pixels_in_target_percent)
    ra_deg = np.degrees(phi)
    dec_deg = np.degrees(0.5 * np.pi - theta)
    
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ'])
    nside_sky = ah.level_to_nside(level)
    ra_sky, dec_sky = ah.healpix_to_lonlat(ipix, nside_sky, order='nested')
    
    matched_indices = []
    for ra, dec in zip(ra_deg, dec_deg):
        matched = np.where((np.abs(ra_sky.deg - ra) < 1e-6) & (np.abs(dec_sky.deg - dec) < 1e-6))[0]
        if len(matched) > 0:
            matched_indices.extend(matched)
    
    matched_indices = np.array(matched_indices)
    
    volumes = np.zeros(len(matched_indices))
    
     # Loop over each matched pixel to calculate volumes
    for i, idx in enumerate(matched_indices):
        pixel_area_deg2 =  hp.nside2pixarea(nside, degrees=True)
        #Convert square degrees to square arcseconds
        area_arcsec2 = pixel_area_deg2 * (3600**2)
        #Convert square arcseconds to square parsecs using the conversion factor
        area_pc2 = area_arcsec2 / (0.2679**2)
        #Convert square parsecs to square megaparsecs (Mpc^2)
        area_mpc2 = area_pc2 / 10**12
        
        dist = 1000
        # Calculate the volume for this pixel
        volumes[i] = (area_mpc2 * dist)*10e-9    # Gpc^3
    
    # Create a structured array to store pixel index, volume, and corresponding distances
    pixel_volumes = np.zeros(len(matched_indices), dtype=[('pixel_index', int), ('volume_Gpc3', float)])
    pixel_volumes['pixel_index'] = matched_indices
    pixel_volumes['volume_Gpc3'] = volumes

    return pixel_volumes

def calculate_expected_transients_poisson(volumes, transients_rates, time_window_days=2):
    total_expected_transients = {transient: 0 for transient in transients_rates}
    time_window_years = time_window_days / 365.25
    
    for volume in volumes:
        for transient, rate in transients_rates.items():
            T = time_window_years
            expected_events = rate * volume * T
            
            sampled_events = np.random.poisson(expected_events)
            total_expected_transients[transient] += sampled_events
    
    return total_expected_transients

def plot_skymap_with_90_percent_region(m, nside, pixels_in_target_percent):
    theta, phi = hp.pix2ang(nside, np.arange(len(m)))
    ra_rad = phi
    dec_rad = 0.5 * np.pi - theta

    ra_rad = ra_rad - np.pi

    theta_90, phi_90 = hp.pix2ang(nside, pixels_in_target_percent)
    ra_90 = phi_90 - np.pi
    dec_90 = 0.5 * np.pi - theta_90
    
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(111, projection='aitoff')
    ax.grid(True)

    sc = ax.scatter(ra_rad, dec_rad, c=m, s=5, cmap='Reds', alpha=0.7, linewidth=0)

    ax.scatter(ra_90, dec_90, color='blue', marker='o', s=2, label='90% Probability Region')

    cbar = plt.colorbar(sc, ax=ax, orientation="horizontal", fraction=0.06, pad=0.1)
    cbar.set_label('Probability Density')

    plt.title("Skymap in Aitoff Projection with 90% Region")
    plt.legend(loc=1)
    plt.show()

In [51]:
def get_transients (gwevent):
    """
    predict contaminant rates given a skymap
    """
    print(f"Calculating expected transients for event {gwevent[0]}")
    skymap_bytes = BytesIO(gwevent[1])
    skymap = gwevent[2]

    transients_rates = {
        'SNIa': 2.35e4,  # per Gpc^3 per year
        'CCSN': 1.01e5,
        'SLSN': 5.6,
        'KN': 5e3,
        'GRB_on_axis': 1,
        'GRB_off_axis': 7,
        'CV': 1e6,  # Use a refined rate near the galactic plane
    }

    # Calculate volumes for the 90% area
    pixel_volumes_in_90_percent = calculate_pixel_volumes(skymap, skymap_bytes, target_probability=0.1)

    # Run multiple simulations
    num_simulations = 10
    all_simulations = {transient: [] for transient in transients_rates}

    time_window_days = 200  # Define the time window

    for _ in range(num_simulations):
        # Calculate expected transients in the 90% probability region with time window integration
        total_expected_transients = calculate_expected_transients_poisson(pixel_volumes_in_90_percent['volume_Gpc3'], transients_rates, time_window_days)
        
        # Store the results of each simulation
        for transient, count in total_expected_transients.items():
            all_simulations[transient].append(count)

    # Calculate mean and standard deviation for each transient type
    mean_transients = {transient: np.mean(all_simulations[transient]) for transient in transients_rates}
    std_transients = {transient: np.std(all_simulations[transient]) for transient in transients_rates}

    # Print the results
    print(f'Mean and standard deviation of sampled transients over {time_window_days} days with {num_simulations} simulations:')
    for transient in transients_rates:
        print(f"{transient}: Mean = {mean_transients[transient]:.15f}, Std = {std_transients[transient]:.15f}")

In [52]:
get_transients(O4a_params[0])

Calculating expected transients for event S230518h
Mean and standard deviation of sampled transients over 200 days with 10 simulations:
SNIa: Mean = 0.000000000000000, Std = 0.000000000000000
CCSN: Mean = 0.000000000000000, Std = 0.000000000000000
SLSN: Mean = 0.000000000000000, Std = 0.000000000000000
KN: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.000000000000000, Std = 0.000000000000000
CV: Mean = 0.000000000000000, Std = 0.000000000000000


In [53]:
get_transients(O4a_params[1])

Calculating expected transients for event S230529ay
Mean and standard deviation of sampled transients over 200 days with 10 simulations:
SNIa: Mean = 0.000000000000000, Std = 0.000000000000000
CCSN: Mean = 0.000000000000000, Std = 0.000000000000000
SLSN: Mean = 0.000000000000000, Std = 0.000000000000000
KN: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.000000000000000, Std = 0.000000000000000
CV: Mean = 1.500000000000000, Std = 1.360147050873544


In [54]:
get_transients(O4a_params[2])

Calculating expected transients for event S230627c
Mean and standard deviation of sampled transients over 200 days with 10 simulations:
SNIa: Mean = 0.000000000000000, Std = 0.000000000000000
CCSN: Mean = 0.000000000000000, Std = 0.000000000000000
SLSN: Mean = 0.000000000000000, Std = 0.000000000000000
KN: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.000000000000000, Std = 0.000000000000000
CV: Mean = 0.000000000000000, Std = 0.000000000000000


In [55]:
get_transients(O4a_params[3])

Calculating expected transients for event S230731an
Mean and standard deviation of sampled transients over 200 days with 10 simulations:
SNIa: Mean = 0.000000000000000, Std = 0.000000000000000
CCSN: Mean = 0.000000000000000, Std = 0.000000000000000
SLSN: Mean = 0.000000000000000, Std = 0.000000000000000
KN: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.000000000000000, Std = 0.000000000000000
CV: Mean = 0.000000000000000, Std = 0.000000000000000


In [56]:
get_transients(O4a_params[4])

Calculating expected transients for event S231113bw
Mean and standard deviation of sampled transients over 200 days with 10 simulations:
SNIa: Mean = 0.000000000000000, Std = 0.000000000000000
CCSN: Mean = 0.000000000000000, Std = 0.000000000000000
SLSN: Mean = 0.000000000000000, Std = 0.000000000000000
KN: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.000000000000000, Std = 0.000000000000000
CV: Mean = 0.100000000000000, Std = 0.300000000000000


# My rates calculation

In [ ]:
# get volume per pixel in 90% region of given skymap

#to do - look at stats: total volume, avg volume per pixel, max volume, min volume   
#could check how different 90% volume is from 90% area, which im currently using

In [12]:
#https://github.com/lpsinger/ligo.skymap/blob/a8da314dcb078f7866f4178cbd523b9844cceae0/ligo/skymap/distance.py#L701

def parameters_to_marginal_moments(prob, distmu, distsigma):
    """Calculate the marginal (integrated all-sky) mean and standard deviation
    of distance from the ansatz parameters.

    Parameters
    ----------
    prob : `numpy.ndarray`
        Marginal probability (pix^-2)
    distmu : `numpy.ndarray`
        Distance location parameter (Mpc)
    distsigma : `numpy.ndarray`
        Distance scale parameter (Mpc)

    Returns
    -------
    distmean : float
        Mean distance (Mpc)
    diststd : float
        Std. deviation of distance (Mpc)

    """
    good = np.isfinite(prob) & np.isfinite(distmu) & np.isfinite(distsigma)
    prob = prob[good]
    distmu = distmu[good]
    distsigma = distsigma[good]
    distmean, diststd, _ = parameters_to_moments(distmu, distsigma)
    rbar = (prob * distmean).sum()
    r2bar = (prob * (np.square(diststd) + np.square(distmean))).sum()
    return rbar, np.sqrt(r2bar - np.square(rbar))

In [57]:
# adapted from https://github.com/lpsinger/ligo.skymap/blob/a8da314dcb078f7866f4178cbd523b9844cceae0/ligo/skymap/postprocess/crossmatch.py

def get_skymap_voxels(skymap, contour=0.9, cosmology=True):
    """
    Determine the volume per pixel in the 90% region of a skymap
    This volume is associated with a sky position and can be used with positionally nonuniform rates
    """
    # get specified countour of skymap, drop other pixels
    skymap.sort('PROBDENSITY', reverse = True)
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ']) # ipix = sky location
    nside = ah.level_to_nside(level) # nside = multi-order pixel resolution
    pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # pixel area in steradians
    prob = pixel_area * skymap['PROBDENSITY']
    cumprob = np.cumsum(prob)
    i = cumprob.searchsorted(contour)
    dA = pixel_area[:i].to_value(u.deg**2) #areas in deg**2 per pixel for only pixels in countour
    #convert dA to mpc^2 ?
    d = skymap.meta['DISTMEAN']

    #get ra and dec in degrees associated with each pixel
    ra, dec = ah.healpix_to_lonlat(ipix[:i], nside[:i])
    radeg = [r.deg for r in ra]
    decdeg = [d.deg for d in dec]

    # Calculate volume of each voxel, defined as the region within the
    # HEALPix pixel and contained within the two centric spherical shells
    # with radii (r - d_r / 2) and (r + d_r / 2).
    r = 500 #mpc
    d_r = 1000 #mpc
    dV = (np.square(r) + np.square(d_r) / 12) * d_r * dA.reshape(-1, 1) #naive euclidean volume

    dV2 = 4/3 * np.pi * 1000**3 * dA.reshape(-1, 1)/(4 * np.pi * d)

    # Calculate probability density per unit volume.
    if cosmology:
        dV *= dVC_dVL_for_DL(r) #comoving volume
    
    V = np.sum(dV2)

    print(f'total volume: {V} Mpc3') 
    return V, dV2, radeg, decdeg

In [58]:
index = 0
print(f'getting volume for {O4a_params[index][0]}')
voxels0 = get_skymap_voxels(O4a_params[index][2])

getting volume for S230518h
total volume: 750825618.1214184 Mpc3


In [63]:
# check to validate assumption comoving volume is necessary at these redshifts

print(f'getting euclidean volume for {O4a_params[index][0]}')
voxels0_euclidean = get_skymap_voxels(O4a_params[index][2], cosmology = False)

getting euclidean volume for S230518h
total volume: 750825618.1214184 Mpc3


In [64]:
index = 1
print(f'getting volume for {O4a_params[index][0]}')
voxels1 = get_skymap_voxels(O4a_params[index][2])

getting volume for S230529ay
total volume: 41475017540.92716 Mpc3


In [65]:
index = 2
print(f'getting volume for {O4a_params[index][0]}')
voxels2 = get_skymap_voxels(O4a_params[index][2])

getting volume for S230627c
total volume: 93330692.82891332 Mpc3


In [67]:
index = 3
print(f'getting volume for {O4a_params[index][0]}')
voxels3 = get_skymap_voxels(O4a_params[index][2])

getting volume for S230731an
total volume: 199260729.54851672 Mpc3


In [68]:
index = 4
print(f'getting volume for {O4a_params[index][0]}')
voxels4 = get_skymap_voxels(O4a_params[index][2])

getting volume for S231113bw
total volume: 481454999.8597008 Mpc3


In [ ]:
# finish this code

def plotter (ra, dec, url):
    center = SkyCoord(ra=0*u.degree, dec=0*u.degree, frame='icrs')

    fig = plt.figure(figsize=(8, 6), dpi=100)

    ax = plt.axes(
        [0.05, 0.05, 0.9, 0.9],
        projection='astro globe',
        center=center)

    ax_inset = plt.axes(
        [0.7, 0.45, 0.7, 0.7],
        projection='astro zoom',
        center=center,
        radius=7*u.deg)

    for key in ['ra', 'dec']:
        ax_inset.coords[key].set_ticklabel_visible(False)
        ax_inset.coords[key].set_ticks_visible(False)
    ax.grid()
    ax.mark_inset_axes(ax_inset)
    ax.connect_inset_axes(ax_inset, 'upper left')
    ax.connect_inset_axes(ax_inset, 'lower left')
    ax_inset.scalebar((0.1, 0.1), 5 * u.deg).label()

    ax.imshow_hpx(url, cmap='cylon')
    ax_inset.imshow_hpx(url, cmap='cylon')

    ax_inset.plot(
        ra, dec, 'X', color='blue', 
        transform=ax_inset.get_transform('world'),
        markersize=1,
        markeredgewidth=1)

In [ ]:
# use volume time rates to get rates per time

In [45]:
def calculate_expected_transients_poisson(volumes, transients_rates, time_window_days):
    total_expected_transients = {transient: 0 for transient in transients_rates}
    time_window_years = time_window_days / 365.25
    
    for volume in volumes:
        volume_gpc = volume * 10e-9 #convert Mpc^3 to Gpc^3

        for transient, rate in transients_rates.items():
            T = time_window_years
            expected_events = rate * volume_gpc * T
            
            sampled_events = np.random.poisson(expected_events)
            total_expected_transients[transient] += sampled_events
    
    return total_expected_transients

In [46]:
def get_transients (gwevent, volume, rates, time_window_days=2, num_simulations=10):    
    """
    predict contaminant rates given a skymap
    """
    print(f"Calculating expected transients for event {gwevent[0]}")

    # Run multiple simulations
    all_simulations = {transient: [] for transient in rates}
    
    for _ in range(num_simulations):
        total_expected_transients = calculate_expected_transients_poisson(volume, rates, time_window_days)
        
        # Store the results of each simulation
        for transient, count in total_expected_transients.items():
            all_simulations[transient].append(count)

    # Calculate mean and standard deviation for each transient type
    mean_transients = {transient: np.mean(all_simulations[transient]) for transient in rates}
    std_transients = {transient: np.std(all_simulations[transient]) for transient in rates}

    # Print the results
    print(f'Mean and standard deviation of sampled transients over {time_window_days} days with {num_simulations} simulations:')
    for transient in rates:
        print(f"{transient}: Mean = {mean_transients[transient]:.15f}, Std = {std_transients[transient]:.15f}")

In [47]:
transients_rates = {
    'SNIa': 2.35e4,  # per Gpc^3 per year
    'CCSN': 1.01e5,
    'SLSN': 5.6,
    'KN': 5e3,
    'GRB_on_axis': 1,
    'GRB_off_axis': 7,
    'CV': 1e6,  # Use a refined rate near the galactic plane
}

In [69]:
transients0 = get_transients(O4a_params[0], voxels0[1], transients_rates)

Calculating expected transients for event S230518h
Mean and standard deviation of sampled transients over 2 days with 10 simulations:
SNIa: Mean = 612.899999999999977, Std = 31.554555930958685
CCSN: Mean = 2651.699999999999818, Std = 55.020087240934103
SLSN: Mean = 0.200000000000000, Std = 0.400000000000000
KN: Mean = 130.000000000000000, Std = 10.158740079360236
GRB_on_axis: Mean = 0.000000000000000, Std = 0.000000000000000
GRB_off_axis: Mean = 0.200000000000000, Std = 0.400000000000000
CV: Mean = 26484.700000000000728, Std = 130.975608416223821


In [ ]:
# next steps: add galactic latitude consideration to rates
